### Leitura Dos Dados de Belo Horizonte

In [1]:
import pandas as pd
from pyproj import Proj, transform
import pandas as pd
import geopandas as gpd 
from shapely import wkt
from pyproj import Transformer
import numpy as np
from pyDOE import lhs

In [ ]:
# Leitura do arquivo csv
bh = pd.read_csv('..\csv\BaseBh.csv')

# Convert GEOMETRIA to a geometry object
bh['GEOMETRIA'] = bh['GEOMETRIA'].apply(wkt.loads)

# Transform the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(bh, geometry='GEOMETRIA')

# Set the current SRID
gdf.crs = "EPSG:31983"

# Define the transformer
transformer = Transformer.from_crs("EPSG:31983", "EPSG:4326")

# Transform the coordinates
gdf['GEOMETRIA'] = gdf['GEOMETRIA'].apply(lambda geom: transformer.transform(geom.x, geom.y))

# Split the coordinates into two columns
gdf[['LONGITUDE', 'LATITUDE']] = pd.DataFrame(gdf['GEOMETRIA'].tolist(), index=gdf.index)

gdf.to_csv('..\csv\baseBHtransformada.csv')

In [2]:
gdf = pd.read_csv('..\csv\baseBHtransformada.csv') 

In [3]:
import pandas as pd
import numpy as np
from scipy.spatial import distance

# Definir o número de amostras desejado
n_samples = 5000

# Extrair as colunas de latitude e longitude do dataframe
coordinates = gdf[['LATITUDE', 'LONGITUDE']].values

# Gerar as amostras usando o Latin Hypercube Sampling
samples = lhs(n=len(coordinates[0]), samples=n_samples)

# Redimensionar as amostras para o intervalo de latitude e longitude
for i in range(len(coordinates[0])):
    samples[:, i] = coordinates[:, i].min() + \
                    samples[:, i] * (coordinates[:, i].max() - coordinates[:, i].min())

# Criar um dataframe vazio para armazenar as amostras
sampled_df = pd.DataFrame(columns=gdf.columns)

# Encontrar o ponto mais próximo para cada amostra
for sample in samples:
    sample_point = sample.reshape(1, -1)
    distances = distance.cdist(sample_point, coordinates)
    closest_idx = np.argmin(distances)
    closest_point = gdf.iloc[closest_idx]
    sampled_df = sampled_df.append(closest_point)

# Redefinir os índices do dataframe resultante
sampled_df.reset_index(drop=True, inplace=True)

# Verificar o dataframe resultante
print(sampled_df.head())


C:\Users\analu\AppData\Local\Temp\ipykernel_5512\2227355403.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(closest_point)
C:\Users\analu\AppData\Local\Temp\ipykernel_5512\2227355403.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(closest_point)
C:\Users\analu\AppData\Local\Temp\ipykernel_5512\2227355403.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(closest_point)
C:\Users\analu\AppData\Local\Temp\ipykernel_5512\2227355403.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(closest_point)
C:\Users

  Unnamed: 0        IDEND  ID_EDC ID_LOGRADOURO SIGLA_TIPO_LOGRADOURO  \
0     751182  30319300138  834897        303193                   RUA   
1     373201  08009201156  372141         80092                   ROD   
2     386855  00060600759  417916           606                   RUA   
3     338072  01367400178  150435         13674                   RUA   
4     149347  04838000138  158873         48380                   AVE   

  DESC_TIPO_LOGRADOURO                  NOME_LOGRADOURO NUMERO_IMOVEL  \
0                  RUA                             TRES           138   
1              RODOVIA  PRESIDENTE JUSCELINO KUBITSCHEK          1156   
2                  RUA                      ABEL ARAUJO           759   
3                  RUA               CARMO DO PARANAIBA           178   
4              AVENIDA           NOSSA SENHORA DA SAUDE           138   

  LETRA_IMOVEL ID_BAIRRO_POPULAR  ... ID_BAIRRO_OFICIAL NUM_BAIRRO_OFICIAL  \
0          NaN               115  ...       

C:\Users\analu\AppData\Local\Temp\ipykernel_5512\2227355403.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(closest_point)
C:\Users\analu\AppData\Local\Temp\ipykernel_5512\2227355403.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(closest_point)
C:\Users\analu\AppData\Local\Temp\ipykernel_5512\2227355403.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(closest_point)
C:\Users\analu\AppData\Local\Temp\ipykernel_5512\2227355403.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(closest_point)
C:\Users

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# Definir o número de amostras desejado
n_samples = 85000

# Extrair as colunas de latitude e longitude do dataframe
coordinates = gdf[['LATITUDE', 'LONGITUDE']].values

# Gerar as amostras usando o Latin Hypercube Sampling
samples = lhs(n=len(coordinates[0]), samples=n_samples)

# Redimensionar as amostras para o intervalo de latitude e longitude
for i in range(len(coordinates[0])):
    samples[:, i] = coordinates[:, i].min() + \
                    samples[:, i] * (coordinates[:, i].max() - coordinates[:, i].min())

# Criar um KD-Tree a partir das coordenadas originais
kdtree = KDTree(coordinates)

# Encontrar o ponto mais próximo para cada amostra
_, closest_indices = kdtree.query(samples)

# Criar um dataframe vazio para armazenar as amostras
sampled_df = pd.DataFrame(columns=gdf.columns)

# Adicionar os pontos mais próximos ao dataframe de amostras
for idx in closest_indices:
    closest_point = gdf.iloc[idx]
    sampled_df = sampled_df.append(closest_point)

# Redefinir os índices do dataframe resultante
sampled_df.reset_index(drop=True, inplace=True)

# Verificar o dataframe resultante
#print(sampled_df.head())


In [4]:
sampled_df.to_csv('..\csv\amostraBH5mil.csv')